In [1]:
import numpy as np
import pandas as pd
import os
import itertools

In [2]:
from classes.model import Model
from classes.scenario import Scenario
from classes.setup import Setup

setup = Setup({
    'dir_output_all_runs': 'birds',  # Optional folder name
})

# Update Torquay to have no IVT unit.
services_updates = {
    # 'PL68DH': {'Use_MSU': 0},
    # 'EX25DW': {'Nearest_MT': 'SW170QT'},  # Crazy value to show that it works.
    'TQ27AA': {'Use_IVT': 0}
}
# SCENARIO 1


# Scenario overwrites default values
scenario = Scenario({
    'name': 'drip-and-ship',
    'destination_decision_type': 0,
    'select_lsoa_method': 'nearest',
    "setup": setup,
    "run_duration": 5,  # years
    "mt_hub_postcodes":['PL68DH', 'BS105NB'],
    "services_updates":services_updates,
    "region_type_for_lsoa_selection":'ICB'
})

model = Model(scenario)

model.run()

from classes.map import plot_map_selected_units, plot_map_catchment, plot_map_outcome
plot_map_selected_units(
    setup, col=scenario.region_column_for_lsoa_selection)
# TEMPORARILY commented out for speed of running. 29th Jan 2024
plot_map_catchment(
    setup, col=scenario.region_column_for_lsoa_selection)
plot_map_outcome(
    setup, col=scenario.region_column_for_lsoa_selection,
    outcome='mrs_shift', destination_type=scenario.destination_decision_type
)

# SCENARIO 2

# Scenario overwrites default values
scenario = Scenario({
    'name': 'mothership',
    'destination_decision_type': 1,
    'select_lsoa_method': 'nearest',
    "setup": setup,
    "run_duration": 5,  # years
    "mt_hub_postcodes":['PL68DH', 'BS105NB'],
    "services_updates":services_updates,
    "region_type_for_lsoa_selection":'ICB'
})

model = Model(scenario)

model.run()

plot_map_selected_units(
    setup, col=scenario.region_column_for_lsoa_selection)
# TEMPORARILY commented out for speed of running. 29th Jan 2024
plot_map_catchment(
    setup, col=scenario.region_column_for_lsoa_selection)
plot_map_outcome(
    setup, col=scenario.region_column_for_lsoa_selection,
    outcome='mrs_shift', destination_type=scenario.destination_decision_type
)


/home/anna/stroke-modelling/simpy-pathway-model/classes/units.py:346: DtypeWarning: Columns (17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df_regions = pd.read_csv(path_to_file)
/home/anna/stroke-modelling/simpy-pathway-model/classes/units.py:436: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if stroke_unit in df_stroke_teams.index.values:
/home/anna/stroke-modelling/simpy-pathway-model/classes/scenario.py:381: DtypeWarning: Columns (17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df_regions = pd.read_csv(path_to_file)
/home/anna/stroke-modelling/simpy-pathway-model/classes/map.py:805: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  gdf.plot(
/home/anna/stroke-modelling/simpy-pathway-model/classes/map.py:805: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothi

In [3]:
# Combine data across all scenarios.

# Top directory:
setup.dir_output_this_setup
setup.dir_output_all_runs

# List of directories of all runs:
output_dir_list = setup.list_dir_output

# Files worth comparing:
# (should be the same across all runs):
setup.file_results_all
setup.file_results_summary_by_admitting_unit
setup.file_results_summary_by_lsoa

'results_summary_by_lsoa.csv'

In [6]:
# Group by LSOA
output_dir = setup.dir_output_all_runs
output_filename = setup.file_combined_results_summary_by_lsoa
file_to_merge = setup.file_results_summary_by_lsoa

data = stack_multiple_dataframes(output_dir_list, file_to_merge)

col_to_group = data.columns[0]
cols_to_keep = ['utility_shift_mean', 'mRS shift_mean', 'mRS 0-2_mean']
df = group_data_and_diff(data, col_to_group, cols_to_keep)

# Save to file:
path_to_file = os.path.join(output_dir, output_filename)
df.to_csv(path_to_file)

df.head(3).T

,0,1,2
lsoa,Bath and North East Somerset 003D,Bath and North East Somerset 008D,Bristol 009C
utility_shift_mean_drip-and-ship,0.099447,NaN,NaN
utility_shift_mean_mothership,NaN,0.103109,NaN
utility_shift_mean_diff_drip-and-ship_minus_mothership,NaN,NaN,NaN
mRS shift_mean_drip-and-ship,-0.541483,NaN,NaN
mRS shift_mean_mothership,NaN,-0.56274,NaN
mRS shift_mean_diff_drip-and-ship_minus_mothership,NaN,NaN,NaN
mRS 0-2_mean_drip-and-ship,0.688337,NaN,NaN
mRS 0-2_mean_mothership,NaN,0.692308,NaN
mRS 0-2_mean_diff_drip-and-ship_minus_mothership,NaN,NaN,NaN


In [7]:
# Group by admitting unit
output_dir = setup.dir_output_all_runs
output_filename = setup.file_combined_results_summary_by_admitting_unit
file_to_merge = setup.file_results_summary_by_admitting_unit

data = stack_multiple_dataframes(output_dir_list, file_to_merge)

col_to_group = data.columns[0]
cols_to_keep = ['utility_shift_mean', 'mRS shift_mean', 'mRS 0-2_mean']
df = group_data_and_diff(data, col_to_group, cols_to_keep)

# Save to file:
path_to_file = os.path.join(output_dir, output_filename)
df.to_csv(path_to_file)

df.head(3).T

,0,1,2
closest_ivt_unit,BA13NG,BA214AT,BS105NB
utility_shift_mean_drip-and-ship,0.099447,0.090361,0.132359
utility_shift_mean_mothership,0.098631,0.092582,0.155452
utility_shift_mean_diff_drip-and-ship_minus_mothership,0.000816,-0.002221,-0.023093
mRS shift_mean_drip-and-ship,-0.541483,-0.457094,-0.697247
mRS shift_mean_mothership,-0.527025,-0.476982,-0.807644
mRS shift_mean_diff_drip-and-ship_minus_mothership,-0.014458,0.019889,0.110398
mRS 0-2_mean_drip-and-ship,0.688337,0.466152,0.604142
mRS 0-2_mean_mothership,0.634271,0.521587,0.549263
mRS 0-2_mean_diff_drip-and-ship_minus_mothership,0.054066,-0.055435,0.054879


In [66]:
# Combine selected regions
output_dir = setup.dir_output_all_runs
output_filename = setup.file_combined_selected_regions
file_to_merge = setup.file_selected_regions

df = hstack_multiple_dataframes(output_dir_list, file_to_merge)
                                  
# Save to file:
path_to_file = os.path.join(output_dir, output_filename)
df.to_csv(path_to_file)

df.head(3)

,contains_selected_unit_drip-and-ship,contains_selected_lsoa_drip-and-ship,contains_selected_unit_mothership,contains_selected_lsoa_mothership
ICB22NM,,,,
"NHS Bath and North East Somerset, Swindon and Wiltshire Integrated Care Board",1,1,1,1
"NHS Bristol, North Somerset and South Gloucestershire Integrated Care Board",1,1,1,1
NHS Cornwall and the Isles of Scilly Integrated Care Board,1,1,1,1


In [60]:
# Combine selected LSOA
output_dir = setup.dir_output_all_runs
output_filename = setup.file_combined_selected_lsoas
file_to_merge = setup.file_selected_lsoas

df = merge_multiple_dataframes(output_dir_list, file_to_merge)
                                  
# Save to file:
path_to_file = os.path.join(output_dir, output_filename)
df.to_csv(path_to_file, index=False)

df.head(3)

,LSOA11CD,LSOA11NM,LSOA11BNG_N,LSOA11BNG_E,LSOA11LONG,LSOA11LAT,ICB22NM,drip-and-ship,mothership
1282,E01014057,Herefordshire 023F,224214,364204,-2.52183,51.9153,NHS Herefordshire and Worcestershire Integrate...,1,1
1283,E01014058,Herefordshire 023G,222214,367966,-2.46695,51.89755,NHS Herefordshire and Worcestershire Integrate...,1,1
388,E01014370,Bath and North East Somerset 007A,165047,375084,-2.35942,51.38391,"NHS Bath and North East Somerset, Swindon and ...",1,1


In [8]:
model.results_all

,id,lsoa,stroke_type,closest_ivt_unit,closest_ivt_travel_duration,closest_mt_unit,closest_mt_travel_duration,mt_transfer_unit,mt_transfer_travel_duration,mt_transfer_required,...,time_ambulance_called,time_ambulance_arrival,time_ambulance_leaves_scene,time_admitting_unit_arrival,time_needle,time_transfer_unit_arrival,time_puncture,mRS shift,utility_shift,mRS 0-2
0,2,Torbay 016E,3,EX25DW,42.3,PL68DH,47.7,PL68DH,59.5,True,...,30.0,60.0,80.0,122.3,NaN,NaN,NaN,NaN,NaN,NaN
1,3,Torbay 017B,3,EX25DW,46.6,PL68DH,50.9,PL68DH,59.5,True,...,30.0,60.0,80.0,126.6,NaN,NaN,NaN,NaN,NaN,NaN
2,1,Mid Devon 008B,2,EX25DW,26.2,PL68DH,76.7,PL68DH,59.5,True,...,30.0,60.0,80.0,106.2,136.2,225.7,285.7,-0.461964,0.094999,0.364661
3,4,Cheltenham 007E,1,GL13NN,17.6,BS105NB,47.7,BS105NB,46.6,True,...,30.0,60.0,80.0,97.6,127.6,NaN,NaN,-0.541483,0.099447,0.688337
4,5,West Devon 005D,1,PL68DH,21.9,PL68DH,21.9,PL68DH,0.0,False,...,30.0,60.0,80.0,101.9,131.9,NaN,NaN,-0.532033,0.097816,0.686568
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,75,Cornwall 068D,0,TR13LQ,36.9,PL68DH,102.5,PL68DH,75.6,True,...,30.0,60.0,80.0,116.9,NaN,NaN,NaN,NaN,NaN,NaN
75,76,Plymouth 015C,3,PL68DH,15.5,PL68DH,15.5,PL68DH,0.0,False,...,30.0,60.0,80.0,95.5,NaN,NaN,NaN,NaN,NaN,NaN
76,77,Sedgemoor 011G,3,TA15DA,25.1,BS105NB,55.2,BS105NB,65.9,True,...,30.0,60.0,80.0,135.2,NaN,NaN,NaN,NaN,NaN,NaN
77,78,Exeter 012C,3,EX25DW,12.2,PL68DH,57.4,PL68DH,59.5,True,...,30.0,60.0,80.0,92.2,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
completed_patients_keys = model.pathway.completed_patients[0].keys()
time_cols = [x for x in completed_patients_keys if x[0:4] == 'time']
time_cols.remove('time_onset')

In [10]:
model.results_summary_all

,time_ambulance_called,time_ambulance_arrival,time_ambulance_leaves_scene,time_admitting_unit_arrival,time_needle,time_transfer_unit_arrival,time_puncture,mRS shift,utility_shift,mRS 0-2
statistic,,,,,,,,,,
mean,30.0,60.0,80.0,108.116456,136.179167,224.109091,230.200000,-0.564027,0.107481,0.584016
std,0.0,0.0,0.0,16.046530,14.247478,9.845248,75.663036,0.159033,0.031688,0.140094


In [11]:
model.results_summary_by_admitting_unit

closest_ivt_unit time_ambulance_called      time_ambulance_arrival       \
                                     mean  std                   mean  std   
0            BA13NG                  30.0  0.0                   60.0  0.0   
1           BA214AT                  30.0  0.0                   60.0  0.0   
2           BS105NB                  30.0  0.0                   60.0  0.0   
3           BS234TQ                  30.0  0.0                   60.0  0.0   
4            DT12JY                  30.0  NaN                   60.0  NaN   
5            EX25DW                  30.0  0.0                   60.0  0.0   
6           EX314JB                  30.0  NaN                   60.0  NaN   
7            GL13NN                  30.0  0.0                   60.0  0.0   
8            PL68DH                  30.0  0.0                   60.0  0.0   
9            SN36BB                  30.0  0.0                   60.0  0.0   
10           TA15DA                  30.0  0.0                   60.0  0.0   
11           TR13LQ                  30.0  0.0                   60.0  0.0   

   time_ambulance_leaves_scene      time_admitting_unit_arrival             \
                          mean  std                        mean        std   
0                         80.0  0.0                  104.933333  11.999611   
1                         80.0  0.0                  105.500000   4.467662   
2                         80.0  0.0                   99.870000   3.416967   
3                         80.0  0.0                   98.666667  11.337695   
4                         80.0  NaN                  119.100000        NaN   
5                         80.0  0.0                  109.871429   9.717821   
6                         80.0  NaN                  105.100000        NaN   
7                         80.0  0.0                  109.818182  12.585215   
8                         80.0  0.0                  104.715385  15.355121   
9                         80.0  0.0                  105.125000  16.842679   
10                        80.0  0.0                  111.037500  16.467452   
11                        80.0  0.0                  131.980000  39.485276   

   time_needle  ... time_transfer_unit_arrival            time_puncture  \
          mean  ...                       mean        std          mean   
0   134.933333  ...                     210.60        NaN        270.60   
1   133.000000  ...                     238.60        NaN        298.60   
2   129.900000  ...                        NaN        NaN        131.26   
3   134.050000  ...                        NaN        NaN           NaN   
4   149.100000  ...                        NaN        NaN           NaN   
5   136.600000  ...                     228.10   1.600000        288.10   
6          NaN  ...                        NaN        NaN           NaN   
7   137.100000  ...                     216.55   0.777817        276.55   
8   143.920000  ...                        NaN        NaN        131.90   
9   121.700000  ...                        NaN        NaN           NaN   
10  134.920000  ...                     223.50  13.953853        283.50   
11  150.533333  ...                        NaN        NaN           NaN   

              mRS shift           utility_shift             mRS 0-2            
          std      mean       std          mean       std      mean       std  
0         NaN -0.527025  0.024615      0.098631  0.004771  0.634271  0.127487  
1         NaN -0.476982  0.077854      0.092582  0.007401  0.521587  0.233318  
2    3.620497 -0.807644  0.220673      0.155452  0.046419  0.549263  0.115168  
3         NaN -0.527284  0.020081      0.096992  0.003473  0.685675  0.003765  
4         NaN -0.494072       NaN      0.091235       NaN  0.679437       NaN  
5    1.600000 -0.490285  0.039548      0.095178  0.002450  0.524303  0.172283  
6         NaN       NaN       NaN           NaN       NaN       NaN       NaN  
7    0.777817 -0.512624  0.030528      0.09772

In [12]:
model.results_all[np.isnan(model.results_all['time_puncture']) == False]

,id,lsoa,stroke_type,closest_ivt_unit,closest_ivt_travel_duration,closest_mt_unit,closest_mt_travel_duration,mt_transfer_unit,mt_transfer_travel_duration,mt_transfer_required,...,time_ambulance_called,time_ambulance_arrival,time_ambulance_leaves_scene,time_admitting_unit_arrival,time_needle,time_transfer_unit_arrival,time_puncture,mRS shift,utility_shift,mRS 0-2
2,1,Mid Devon 008B,2,EX25DW,26.2,PL68DH,76.7,PL68DH,59.5,True,...,30.0,60.0,80.0,106.2,136.2,225.7,285.7,-0.461964,0.094999,0.364661
6,7,Bristol 007B,2,BS105NB,16.5,BS105NB,16.5,BS105NB,0.0,False,...,30.0,60.0,80.0,96.5,126.5,NaN,126.5,-0.983666,0.192008,0.469035
22,24,West Devon 005D,2,PL68DH,21.9,PL68DH,21.9,PL68DH,0.0,False,...,30.0,60.0,80.0,101.9,131.9,NaN,131.9,-0.965190,0.188667,0.465395
25,25,East Devon 012B,2,EX25DW,29.4,PL68DH,72.4,PL68DH,59.5,True,...,30.0,60.0,80.0,109.4,139.4,228.9,288.9,-0.452001,0.093097,0.362656
34,35,South Gloucestershire 005C,2,BS105NB,25.1,BS105NB,25.1,BS105NB,0.0,False,...,30.0,60.0,80.0,105.1,135.1,NaN,135.1,-0.954264,0.186687,0.463239
36,31,Stroud 009B,2,GL13NN,29.4,BS105NB,40.2,BS105NB,46.6,True,...,30.0,60.0,80.0,109.4,139.4,216.0,276.0,-0.492295,0.100778,0.370769
38,39,Bristol 033D,2,BS105NB,18.7,BS105NB,18.7,BS105NB,0.0,False,...,30.0,60.0,80.0,98.7,128.7,NaN,128.7,-0.976133,0.190646,0.467552
39,40,Bristol 047A,2,BS105NB,21.9,BS105NB,21.9,BS105NB,0.0,False,...,30.0,60.0,80.0,101.9,131.9,NaN,131.9,-0.965190,0.188667,0.465395
42,42,Mid Devon 002D,2,TA15DA,33.7,PL68DH,81.0,BS105NB,65.9,True,...,30.0,60.0,80.0,113.7,143.7,239.6,299.6,-0.418843,0.086758,0.355985
46,46,Wiltshire 017D,2,BA13NG,24.1,BS105NB,36.9,BS105NB,33.7,True,...,30.0,60.0,80.0,116.9,146.9,210.6,270.6,-0.509264,0.104005,0.374188


In [13]:
model.results_all[np.isnan(model.results_all['time_needle']) == False]

,id,lsoa,stroke_type,closest_ivt_unit,closest_ivt_travel_duration,closest_mt_unit,closest_mt_travel_duration,mt_transfer_unit,mt_transfer_travel_duration,mt_transfer_required,...,time_ambulance_called,time_ambulance_arrival,time_ambulance_leaves_scene,time_admitting_unit_arrival,time_needle,time_transfer_unit_arrival,time_puncture,mRS shift,utility_shift,mRS 0-2
2,1,Mid Devon 008B,2,EX25DW,26.2,PL68DH,76.7,PL68DH,59.5,True,...,30.0,60.0,80.0,106.2,136.2,225.7,285.7,-0.461964,0.094999,0.364661
3,4,Cheltenham 007E,1,GL13NN,17.6,BS105NB,47.7,BS105NB,46.6,True,...,30.0,60.0,80.0,97.6,127.6,NaN,NaN,-0.541483,0.099447,0.688337
4,5,West Devon 005D,1,PL68DH,21.9,PL68DH,21.9,PL68DH,0.0,False,...,30.0,60.0,80.0,101.9,131.9,NaN,NaN,-0.532033,0.097816,0.686568
5,6,North Somerset 010A,1,BS234TQ,23.0,BS105NB,30.5,BS105NB,43.4,True,...,30.0,60.0,80.0,110.5,140.5,NaN,NaN,-0.513085,0.094536,0.683013
6,7,Bristol 007B,2,BS105NB,16.5,BS105NB,16.5,BS105NB,0.0,False,...,30.0,60.0,80.0,96.5,126.5,NaN,126.5,-0.983666,0.192008,0.469035
7,8,Cornwall 043A,1,TR13LQ,9.0,PL68DH,77.8,PL68DH,75.6,True,...,30.0,60.0,80.0,89.0,119.0,NaN,NaN,-0.560334,0.102695,0.691859
12,12,Cornwall 064D,1,TR13LQ,24.1,PL68DH,85.3,PL68DH,75.6,True,...,30.0,60.0,80.0,165.3,195.3,NaN,NaN,-0.390894,0.073142,0.659870
13,14,Forest of Dean 008A,1,GL13NN,40.2,BS105NB,42.3,BS105NB,46.6,True,...,30.0,60.0,80.0,120.2,150.2,NaN,NaN,-0.491636,0.090811,0.678978
15,16,Swindon 019D,1,SN36BB,12.2,BS105NB,48.8,BS105NB,49.8,True,...,30.0,60.0,80.0,92.2,122.2,NaN,NaN,-0.553327,0.101489,0.690551
16,17,Cornwall 057D,1,TR13LQ,27.3,PL68DH,93.9,PL68DH,75.6,True,...,30.0,60.0,80.0,107.3,137.3,NaN,NaN,-0.520143,0.095759,0.684338
